In [44]:
import os
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
from datetime import timedelta


In [27]:
pd.set_option('display.max_rows', 8)

In [28]:
my_data = pd.read_excel("ippis_deduc_reports.xlsx", sheet_name = None)
my_data = pd.concat(my_data[frame] for frame in my_data.keys())

deduction = my_data.groupby(['Employee Number', 'Employee Name', 'Ministry Name']).agg(["sum","count"]).reset_index()



In [29]:
deduction_columns = deduction.columns.map(''.join)
deduction.columns = deduction_columns
deduction.columns

Index(['Employee Number', 'Employee Name', 'Ministry Name',
       'Deduction Amountsum', 'Deduction Amountcount', 'Element Namesum',
       'Element Namecount', 'Macro1sum', 'Macro1count', 'Auto_Opensum',
       'Auto_Opencount'],
      dtype='object')

In [30]:
required_columns = ['Employee Number',
                    'Ministry Name',
                    'Employee Name',
                    'Deduction Amountsum',
                    'Deduction Amountcount', 
                    ]

target_deduction = deduction[required_columns]
deduction = target_deduction.rename(columns = {'Deduction Amountsum': 'TotalDeducted', 'Deduction Amountcount': 'Nos Deducted'})

# deduction.head(7)

In [31]:
deduction.to_excel('Cleaned_Deduction_Report.xlsx')

In [32]:
loandisk = pd.read_csv("ippis_loandisk_2020_new.csv")

dateRel = pd.to_datetime(loandisk["Released"])
loandisk["Released"]= dateRel

MatDate = pd.to_datetime(loandisk["Maturity"])
loandisk["Maturity"]= MatDate

_loandisk = loandisk.replace({
                            'Duration' : '[A-Za-z]', 
                            },'', regex=True)
loandisk = _loandisk

loandisk = loandisk[loandisk["IPPIS No"].notnull()]

loandisk.to_excel('Cleaned_loan_disk.xlsx')

In [45]:
ippis_data = pd.read_excel("Cleaned_loan_disk.xlsx")
# ippis_data

In [46]:
file_merge = pd.merge(
                left= ippis_data,
                right= deduction,
                how="outer",
                left_on="IPPIS No",
                right_on="Employee Number"
                )



In [47]:
file_merge.columns

Index(['Unnamed: 0', 'Released', 'Maturity', 'Loan#', 'Name', 'IPPIS No',
       'Principal', 'Duration', 'Due', 'Paid', 'Amortization', 'PendingDue',
       'Last Paid Date', 'Last Payment', 'Balance', 'MINISTRY/AGENCY',
       'Status', 'Employee Number', 'Ministry Name', 'Employee Name',
       'TotalDeducted', 'Nos Deducted'],
      dtype='object')

In [48]:
new_merge = file_merge[['Released',
                        'Maturity',
                        'Loan#',
                        'Name',
                        'IPPIS No',
                        'Employee Number',
                        'Principal',
                        'Due',
                        'Paid',
                        'TotalDeducted',
                        'PendingDue',
                        'Last Payment',
                        'Duration',
                        'Nos Deducted',
                        'Balance',
                        'MINISTRY/AGENCY',
                        'Status'
                        ]]


In [49]:
merged_data = new_merge.rename(columns={'Paid':'LoanDiskPaid'})

processingMask = merged_data['Status'] != 'Processing'
merged_data = merged_data[processingMask]

# merged_data.head(5)

In [50]:
merged_data['TotalDeducted'] = merged_data['TotalDeducted'].replace(to_replace = np.nan, value = 0)

merged_data['LoanDiskPaid'] = merged_data['LoanDiskPaid'].replace(to_replace = np.nan, value = 0)

In [73]:
merged_data['Credit_Diff'] = merged_data['TotalDeducted'] - merged_data['LoanDiskPaid']
merged_data['Credit_Diff'] = merged_data['Credit_Diff'].apply(lambda x: round(x, 3))

merged_data['LoanBalance'] = merged_data['Due'] - merged_data['TotalDeducted']
merged_data['LoanBalance'] = merged_data['LoanBalance'].apply(lambda x: round(x, 0))

merged_data['Amortization'] = merged_data['Due'] / merged_data['Duration']
merged_data['Amortization'] = merged_data['Amortization'].apply(lambda x: round(x, 2))

merged_data['PendingDuration'] = (merged_data['Due'] - merged_data['TotalDeducted']) / merged_data['Amortization']
merged_data['PendingDuration'] = merged_data['PendingDuration'].apply(lambda x: round(x, 0))

merged_data['Deduction Amount'] = merged_data['LoanBalance'] / merged_data['PendingDuration']
merged_data['Deduction Amount'] = merged_data['Deduction Amount'].apply(lambda x: round(x, 0))

merged_data['Start Date'] = ""
merged_data['End Date'] = ""


# merged_data

In [74]:
PMask = merged_data[(merged_data['LoanBalance'] == 0.01) & (merged_data[(merged_data['LoanBalance'] <= 3)]
# update = merged_data[PMask]
# update.to_csv('Update LoanBalance.csv')

SyntaxError: unexpected EOF while parsing (<ipython-input-74-d8dd09c1ff99>, line 3)

In [75]:
current_month = dt.datetime.today().strftime('%m/%y')
merged_data['Start Date'] = current_month
# type(merged_data['Start Date'])

In [76]:
NLoan_status = []
for data in merged_data['LoanBalance']:
    if data < -10 :
        NLoan_status.append('Excess Payment')
    elif data == 0 or data < 5:
        NLoan_status.append('fully paid')
    elif data > 5:
        NLoan_status.append('running')
    elif np.nan:
        NLoan_status.append('no deduction')
    else:
        pass
        
NLoan_status
merged_data['Loan status'] = NLoan_status

# merged_data.to_csv('csv.csv')

In [77]:
# i=merged_data[(merged_data['Loan status'] != 'Excess Payment') & (merged_data['Loan status'] != 'fully paid')]
gh = merged_data[merged_data['Status'] == 'Past Maturity']
i = gh[(merged_data['Loan status'] != 'Excess Payment') & (merged_data['Loan status'] != 'fully paid')]
# i

C:\Users\hp\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [78]:
new_df = i['LoanBalance'] >= 3
final_data = i[new_df]

In [79]:
deductionList = {
    "Employee Number" : final_data['IPPIS No'],
    "Client Name" : final_data['Name'],
   "Start_Date" : final_data['Start Date'],
    "End_Date" : final_data['End Date'],
    "Deduction Amount" : final_data['Deduction Amount'],
    "Duration" : final_data['PendingDuration'],
     "Loan Balance" : final_data['LoanBalance']
}
# deductionList
df = pd.DataFrame(deductionList)
df.to_csv('final_deduction.csv', index=False)

In [80]:
nd = i['TotalDeducted']==0
notdeducted = i[nd]

In [81]:
NotDeducted = {
    "Employee Number" : notdeducted['IPPIS No'],
    "Client Name" : notdeducted['Name'],
    "Principal" : notdeducted['Principal'],
    "Tenor" : notdeducted['Duration'],
   "Ministry Name" : notdeducted['MINISTRY/AGENCY'],
    "Expected Payment" : notdeducted['Due'],
    "Loan Balance": notdeducted['LoanBalance']
   }
# deductionList
dfa = pd.DataFrame(NotDeducted)
dfa.to_csv('not_deducted.csv', index=False)

In [82]:
excess_df = gh['Loan status'] == 'Excess Payment'
excess_data = gh[excess_df]
# excess_data

In [83]:
excessPayment = {
     "Employee Number" : excess_data['IPPIS No'],
    "Client Name" : excess_data['Name'],
   "Start_Date" : excess_data['Start Date'],
    "End_Date" : excess_data['End Date'],
    "Deduction Amount" : excess_data['Deduction Amount'],
    "Loan Balance" : excess_data['LoanBalance']
    
}
nddf = pd.DataFrame(excess_data)
nddf.to_csv('Excess_Deduction.csv', index=False)

In [ ]:
# merged_data["Deduct_Status"] = merged_data["Nos Deducted"].replace(to_replace=[np.nan,1.0,2.0,3.0,4.0], value = ['not_Deducted','once', 'twice', 'thrice', 'check payment'])
# merged_data.head()

In [85]:
# merged_data['Loan status'] = merged_data['Loan status'].where(merged_data['Loan status'] == 'running', 'no deduction')
# merged_data['Loan status'] # merged_data[(merged_data['Loan status'] != 'Excess Payment') & (merged_data['Loan status'] != 'fully paid')]

In [ ]:
# ydata = pd.read_excel("ippis_deduc_reports.xlsx", sheet_name = None)
# ydata = pd.concat(ydata[frame] for frame in my_data.keys())

# sorti = my_data.groupby(['Employee Number', 'Employee Name', 'Ministry Name']).agg(["sum","count"]).reset_index()

In [ ]:
# deduction
# df = pd.DataFrame(deduction)|
# df.rename(columns={0:"Employee Number"})
# df.columns

johnson = my_data.groupby(["Employee Number","Employee Name", "Month", "Ministry Name"])["Deduction Amount"].agg(["sum","count"]).reset_index()

In [ ]:
# deduction = my_data.groupby('Employee Number').sum().reset_index()

In [ ]:
# deduction = my_data.groupby('Employee Number')['Employee Number'].transform('count').sum()

In [ ]:
# pd.set_option("display.max_columns", 20)


In [ ]:
# johnson

In [ ]:
# deduction.get_group("Ministry Name")

In [ ]:
# current_Month = datetime.datetime.now()
# month = current_Month.strftime("%B")
# month = date.month
# month

In [ ]:
# current_Month = datetime.datetime.now()
# dmonth = current_Month.month,year
# # dmonth
# type(dmonth)

In [ ]:
# my_data.groupby(['Element Name']).get_group("LIBERTY ASSURED LIMITED")

In [ ]:
# deduc_309344_mask = johnson["Employee Number"]==309344
# search = johnson[deduc_309344_mask]
# search

new_deduction = my_data[deduction]
new_deduction

In [ ]:
# done = deduction.get("Ministry Name")
# done

deduc_309344_mask = deduction["Employee Number"]==309344
search = deduction[deduc_309344_mask]
search

In [ ]:
# ippis_data.drop(columns = ['Remita Mandate Status', 'Mobile','Principal After Deduct', 'Account Number', 'Bank Code'])

In [52]:
# valid_data = merged_data[(merged_data['Loan status'] != 'Excess Payment') & (merged_data['Loan status'] != 'fully paid')]
# merged_data = merged_data[valid_data]
# new_df = merged_data['LoanBalance'] >= 3
# final_data = merged_data[new_df]

# final_data

In [118]:


# columns = deduction.columns
# my_data2.columns
# columns.apply(tuple)
# data

In [120]:
# valid_data = merged_data[(merged_data['Loan status'] != 'Excess Payment') & (merged_data['Loan status'] != 'fully paid')]
# new_df = valid_data(valid_data['LoanBalance'] >= 3) & (valid_data['LoanBalance'] == np.nan)]
# final_data = valid_data[new_df]
# new_df
# final_data
# # valid_data